In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow as tf2
import numpy as np
import matplotlib
import os
import glob
import random
from sklearn.preprocessing import OneHotEncoder

tf.set_random_seed(777)  
import matplotlib.pyplot as plt
%matplotlib inline

num_sector=9
num_=0
traing_predict_=[]
trainY_=[]
test_predict_=[]
testY_=[]
days_later = 0
time_series= 'data size'+seq_length
seq_length = 100 
data_dim = 16
num_classes= 3
# num_classes= 10
# num_classes= 9
output_dim = num_classes
acc_=[]
iterations=60000

xy = np.loadtxt('Input_data_path', delimiter=',', dtype=np.float32)
xy_te = np.loadtxt('Input_data_path', delimiter=',', dtype=np.float32)

xy_data=xy.reshape(-1,21)
xy_te_data=xy_te.reshape(-1,21)

y_class=(xy_data[:,-1].reshape(-1,1))
enc = OneHotEncoder(categories='auto')
enc.fit(y_class)
y_class_onehot = enc.transform(y_class).toarray()

y_class_te=(xy_te_data[:,-1].reshape(-1,1))
enc_te = OneHotEncoder(categories='auto')
enc_te.fit(y_class_te)
y_class_onehot_te = enc_te.transform(y_class_te).toarray()

x_data_training = xy_data[:, :-5]
y_data_training = y_class_onehot[:, :]
x_data_test = xy_te_data[:, :-5]
y_data_test = y_class_onehot_te[:, :]

trainX__=x_data_training.reshape(-1,(time_series-seq_length)*num_sector,seq_length,data_dim)
trainY__=y_data_training.reshape(-1,time_series-seq_length,seq_length,y_class_onehot.shape[1])
trainY__=np.mean(trainY__,axis=2)
trainY__=trainY__.reshape(-1,(time_series-seq_length)*num_sector,y_class_onehot.shape[1])

testX__=x_data_test.reshape(-1,(time_series-seq_length)*num_sector,seq_length,data_dim)
testY__=y_data_test.reshape(-1,time_series-seq_length,seq_length,y_class_onehot_te.shape[1])
testY__=np.mean(testY__,axis=2)
testY__=testY__.reshape(-1,(time_series-seq_length)*num_sector,y_class_onehot.shape[1])

hidden_dim = 64
learning_rate = 0.000001
num_signal = 16



graph = tf.Graph()
with graph.as_default():
        

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, num_classes])

    # build a LSTM network
    cell = tf.keras.layers.LSTMCell(units=hidden_dim, activation='tanh')

    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

    Y_pred_1 = tf.keras.layers.Dense(units=256, activation='relu')(outputs[:,-1])  
    Y_pred_2 = tf.keras.layers.Dense(units=512, activation='relu')(Y_pred_1)
    Y_pred_3 = tf.keras.layers.Dense(units=256, activation='relu')(Y_pred_2)
    Y_pred_ = tf.keras.layers.Dense(output_dim,activation='softmax')(Y_pred_3)
    Y_pred = tf.nn.softmax(Y_pred_)
   
    # cost/loss
    loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y_pred, labels=Y))  

    # optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    train = optimizer.minimize(loss)
                           
    prediction = tf.argmax(Y_pred, 1)
    label = tf.argmax(Y, 1)
    prediction_corrected = tf.equal(label, prediction)
    accuracy = tf.reduce_mean(tf.cast(prediction_corrected, tf.float32))    


with graph.as_default():
    saver = tf.train.Saver()
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())

        # Training step
    for i in range(iterations):
        ran_=np.random.randint(len(trainX__))
        _, step_loss, acc = sess.run([train, loss, accuracy], feed_dict={X: trainX__[ran_], Y: trainY__[ran_]})
        print("[step: {}] loss: {} accuracy: {}" .format(i, step_loss, acc))
        
    saver.save(sess,"save_path")
        
        # Test step
    for j in range(len(testX__)):
        test_predict = sess.run(prediction, feed_dict={X: testX__[j], Y: testY__[j]})
        y_label=  sess.run(label, feed_dict={X: testX__[j], Y: testY__[j]})                   
        y_acc= sess.run(accuracy, feed_dict={X: testX__[j], Y: testY__[j]})
        print("Accuracy: {}".format(y_acc))

        test_predict_.append(test_predict)
        testY_.append(y_label)
        acc_.append(y_acc)   
                
print(np.mean(acc_))               
test_predict__=np.array(test_predict_)
testY__=np.array(testY_)                   